The new plan is as follows:

Use a pre-trained sentence-transformer model to convert each catalog_content entry into a dense numerical vector (an embedding). This is a one-time, computationally expensive step.

Train our tuned LightGBM model on these new embedding features instead of the TF-IDF features.

In [1]:
!pip install -U sentence-transformers

  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 971.0 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 1.3 MB/s eta 0:00:00-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 1.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 MB 1.2 MB/s eta 0:00:0000:0100:02
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13/13 [sentence-transformers]ence-transformers]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import re
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import joblib

# --- 1. Load Data ---
train_df = pd.read_csv('/Users/adityasharma/Github Projects/Amazon/input/train.csv')
train_df = train_df.dropna(subset=['price'])
train_df['catalog_content'] = train_df['catalog_content'].astype(str).fillna('')
print("Training data loaded successfully.")

# --- 2. Generate Semantic Embeddings ---
print("Loading pre-trained sentence-transformer model...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

print("Generating embeddings for all catalog_content... (This will take several minutes)")
def extract_title(text):
    match = re.search(r'^item name:\s*(.*)', text, re.IGNORECASE | re.MULTILINE)
    return match.group(1).strip() if match else text
train_df['title'] = train_df['catalog_content'].apply(extract_title)

embeddings = embedding_model.encode(train_df['title'].tolist(), show_progress_bar=True)
print(f"Embeddings created with shape: {embeddings.shape}")


# --- 3. Create Hold-Out Set ---
X = embeddings
y = train_df['price']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
y_train_log = np.log1p(y_train)

# --- 4. Train LightGBM on the New Embedding Features ---
best_params = {
    'objective': 'regression_l1', 'metric': 'mae', 'n_estimators': 761, 'learning_rate': 0.188,
    'num_leaves': 41, 'max_depth': 17, 'lambda_l1': 0.04, 'lambda_l2': 2.53e-06,
    'feature_fraction': 0.73, 'bagging_fraction': 0.81, 'bagging_freq': 5,
    'min_child_samples': 9, 'random_state': 42,
    'n_jobs': -1, # <-- FIX: Corrected the value from '-नहीं' to '-1'
    'verbose': -1
}

model_v4 = lgb.LGBMRegressor(**best_params)

print("\nTraining V4 model on semantic embeddings...")
model_v4.fit(X_train, y_train_log)
print("Training complete.")

# --- 5. Evaluate the V4 Model ---
print("\nEvaluating V4 model...")
val_preds_log = model_v4.predict(X_val)
val_preds = np.expm1(val_preds_log)
val_preds[val_preds < 0] = 0

def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(np.divide(numerator, denominator, out=np.zeros_like(numerator, dtype=float), where=denominator!=0)) * 100

v4_smape = smape(y_val, val_preds)

print("\n--- Model Performance Comparison ---")
print(f"V1 Model SMAPE (TF-IDF on raw text): 51.9241")
print(f"V4 Model SMAPE (Semantic Embeddings): {v4_smape:.4f}")

Training data loaded successfully.
Loading pre-trained sentence-transformer model...
Generating embeddings for all catalog_content... (This will take several minutes)


Batches:   0%|          | 0/2344 [00:00<?, ?it/s]

Embeddings created with shape: (75000, 384)

Training V4 model on semantic embeddings...
Training complete.

Evaluating V4 model...

--- Model Performance Comparison ---
V1 Model SMAPE (TF-IDF on raw text): 51.9241
V4 Model SMAPE (Semantic Embeddings): 58.8297


: 